In [8]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import pipeline

pipe = pipeline("text-generation", model="skt/kogpt2-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")

In [4]:
from peft import PeftModel

# QLoRA로 파인튜닝된 모델 경로
qlora_model_path ='/content/drive/MyDrive/hatra3'
qlora_model = PeftModel.from_pretrained(model, qlora_model_path)

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
model.load_state_dict(qlora_model.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['transformer.wte.weight', 'transformer.wpe.weight', 'transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.base_layer.weight', 'transformer.h.0.attn.c_attn.base_layer.bias', 'transformer.h.0.attn.c_attn.lora_A.default.weight', 'transformer.h.0.attn.c_attn.lora_B.default.weight', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.1.ln_1.weight', 'transformer.h.1.ln_1.bias', 'transformer.h.1.attn.c_attn.base_layer.weight', 'transformer.h.1.attn.c_attn.base_layer.bias', 'transformer.h.1.attn.c_attn.lora_A.default.weight', 'transformer.h.1.attn.c_attn.lora_B.default.weight', 'transformer.h.1.attn.c_proj.weight', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.ln_2.weight', 'transformer.

In [6]:
input_text = "기흥시 종목수를 알려줘"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# 모델을 통해 텍스트 생성
output = model.generate(input_ids, max_length=100)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

기흥시 종목수를 알려줘.[/INST] An: 해당 시설의 상세 주소는 경기도 시흥시 시흥대로 445이며, 해당 강좌는 주로 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)월에 개최하며, 가격은 110000원 입니다.


In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# QLoRA 모델 경로
qlora_model_path = '/content/drive/MyDrive/hatra3'

base_model = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
q_model = AutoModelForCausalLM.from_pretrained(base_model)

# QLoRA 모델 로드
#model = PeftModel.from_pretrained(q_model, qlora_model_path)

try:
    model = PeftModel.from_pretrained(q_model, qlora_model_path)
except RuntimeError as e:
    print("Error loading QLoRA model:", e)
    # 모델을 직접 로드하는 방법
    state_dict_path = f"{qlora_model_path}/pytorch_model.bin"  # 체크포인트 파일 경로
    state_dict = torch.load(state_dict_path)  # 파일 경로로 수정
    missing_keys, unexpected_keys = q_model.load_state_dict(state_dict, strict=False)
    print("Missing keys:", missing_keys)
    print("Unexpected keys:", unexpected_keys)
    model = q_model  # 기본 모델로 설정


def generate_answer(question):
    # 입력 형식에 맞게 텍스트 생성
    input_text = f"Qu: {question}[/INST] An:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 모델을 통해 답변 생성
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=1024,  # 최대 길이 설정
            do_sample=True,  # 샘플링 사용
            top_k=50,  # 상위 k개 샘플링
            top_p=0.95,  # 누적 확률
            temperature=0.3,  # 온도 조절
            pad_token_id=tokenizer.eos_token_id  # 패딩 토큰 설정
        )

    # 생성된 텍스트 디코딩
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    # "An:" 이후의 답변만 추출
    answer_start = answer.find("An:") + len("An:")
    final_answer = answer[answer_start:].strip()  # 공백 제거
    return final_answer

# 챗봇 사용 예시
while True:
    user_input = input("Q: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = generate_answer(user_input)
    print("A:", response)


Q: 서울 노원구에서 테니스에 대한 강좌를 알려줘.
A: 서울 노원구에는 테니스에 관한 프로그램이 존재하지 않습니다.
Q: 경기 용인시지역에는 어떤 종목들이 존재하는지 알려줘
A: 경기 용인시 지역에는 [태권도, 필라테스, 기타종목]종목들이 존재합니다.
Q: 원포인트특수체육센터의 기타종목 강좌의 정보를 상세히 알려줘
A: 해당 시설의 상세 주소는 경기도 안산시 상록구 상록대로 26이며, 해당 강좌는 주로 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)월에 개최하며, 가격은 110000원 입니다.
Q:  인천광역시 태권도 관련 장애인 시설 정보를 알려줘
A: 인천 남동구에 있는 체육시설의 상세 주소는 인천광역시 남동구 구월로 24이며, 우편번호가 4173이고, 이용 가능한 종목은 태권도입니다. 제공하는 강좌의 강좌명은 태권도이며, 강좌는 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)월에 개최되며, 가격은 110000원입니다.
Q: exit


In [30]:
import gradio as gr

qlora_model_path = '/content/drive/MyDrive/hatra3'

# 기본 모델 로드
base_model = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
q_model = AutoModelForCausalLM.from_pretrained(base_model)

# QLoRA 모델 로드
model = PeftModel.from_pretrained(q_model, qlora_model_path)

def chatbot_response(user_input):
    # 입력 텍스트 포맷
    input_text = f"Qu: {user_input}[/INST] An: "

    # 입력을 토큰화
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 모델을 사용하여 응답 생성
    output = model.generate(
        input_ids,
        max_length=1000,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        top_k=30,
        top_p=0.80
    )

    # 응답 디코딩
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response = generated_text.split('An: ')[1] if 'An: ' in generated_text else generated_text
    return response.strip()

# Gradio 인터페이스 구성
iface = gr.Interface(fn=chatbot_response,
                     inputs="text",
                     outputs="text",
                     title="운동강좌 챗봇",
                     description="입력하세요")

# 인터페이스 실행
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd241f9623fdb6879e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
